Model Training

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [108]:
init_df = pd.read_csv("data\gemstone.csv")
init_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [109]:
init_df=init_df.drop(columns=['id'])
init_df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [110]:
def featureAdd(df):
    for i in ['x','y','z']:
        df[i]=df[i].replace(0,init_df[i].median())
    
    for i in ['x','y','z']:
        df[i]=np.log(df[i])


    df['C/A']=(df['carat'])/(df['depth']*df['table'])
    df['DR']=(df['depth']/df['x'])

    return df

In [111]:
init_df=featureAdd(init_df)

In [112]:
# Seggregation of Independent and Dependent feature
x=init_df.drop(columns=['price'],axis=1)
y=init_df['price']

In [113]:
init_df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price,C/A,DR
0,1.52,Premium,F,VS2,62.2,58.0,1.983756,1.991976,1.515127,13619,0.000421,31.354658
1,2.03,Very Good,J,SI2,62.0,58.0,2.086914,2.094330,1.619388,13387,0.000565,29.708945
2,0.70,Ideal,G,VS1,61.2,57.0,1.738710,1.745716,1.252763,2772,0.000201,35.198504
3,0.32,Ideal,G,VS1,61.6,56.0,1.477049,1.483875,0.996949,666,0.000093,41.704785
4,1.70,Premium,G,VS2,62.6,59.0,2.034706,2.029463,1.562346,14453,0.000460,30.766121


In [114]:
# Seggregation between categorical and numerical feature
cat_feature=x.select_dtypes(include='object').columns
num_feature=x.select_dtypes(exclude='object').columns

In [115]:
imp_num_feature=['carat','y','DR','C/A']

In [116]:
x['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [117]:
# Defining rank to categorical features
cut_rank=['Fair','Good','Very Good','Premium','Ideal']
color_rank=['D','E','F','G','H','I','J']
clarity_rank=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [118]:
from sklearn.impute import SimpleImputer #Handling Missing values
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OrdinalEncoder

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [119]:
# Numerical pipeline
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())

    ]
)

# Categorical pipeline
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_rank,color_rank,clarity_rank])),
        ('scaler',StandardScaler())

    ]
)


preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,imp_num_feature),
    ('cat_pipeline',cat_pipeline,cat_feature)
])

In [120]:
# Train Test Split
from sklearn.model_selection import train_test_split

In [121]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.25)

In [122]:
# Applying the pipelines to train_set and test_test
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [123]:
x_train.head()

,num_pipeline__carat,num_pipeline__y,num_pipeline__DR,num_pipeline__C/A,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.472019,0.688776,-0.680542,0.522983,0.873371,1.527634,-1.315033
1,0.623099,0.833179,-0.696706,0.699169,0.873371,0.911526,0.017494
2,2.651895,2.023529,-1.902376,2.633837,-0.132110,1.527634,0.017494
3,-1.017203,-1.320714,1.613845,-1.023828,-1.137590,0.295418,-0.648770
4,-0.110720,0.085804,-0.022200,-0.313872,-2.143070,-0.936798,-1.315033


In [124]:
# Model Training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [125]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2=r2_score(true,predicted)

    return mae,mse,rmse,r2



In [126]:
# Multiple model training

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
    
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)


    y_pred=model.predict(x_test)

    mae,mse,rmse,r2=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    
    print("RMSE: ",rmse)
    print("MAE: ",mae)
    print("MSE: ",mse)
    print("R2: ",r2)

    r2_list.append(r2)

    print("="*30)
    print("\n")




LinearRegression
RMSE:  1007.0411569018659
MAE:  665.9345863524755
MSE:  1014131.8916942483
R2:  0.9375126290899957


Lasso
RMSE:  1007.2166171267728
MAE:  666.7703556455043
MSE:  1014485.3138163001
R2:  0.9374908524163604


Ridge
RMSE:  1007.0447231936687
MAE:  665.9529755839948
MSE:  1014139.0745122128
R2:  0.9375121865090907


ElasticNet
RMSE:  1498.5287830173622
MAE:  1024.3083455176816
MSE:  2245588.5135314967
R2:  0.8616344446856292




d:\Project_Diamond_Price_Prediction\venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.250e+08, tolerance: 2.365e+08
  model = cd_fast.enet_coordinate_descent(
